In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from tqdm import tqdm
#from oauth2client.client import GoogleCredentials
import numpy as np

from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.model_selection import train_test_split

In [2]:
print(os.getcwd())

/Users/sbezawada/Documents/Workspace/MLE-COURSE/Capstone2


In [3]:
DRIVE_FOLDER = '/Users/sbezawada/Documents/Workspace/MLE-COURSE/Capstone/Data/false/'
entries = [entry for entry in os.listdir(DRIVE_FOLDER) if os.path.isfile(os.path.join(DRIVE_FOLDER, entry))]
false_dict = {'name':[]}
for file in entries:

    false_dict['name'].append(file)
DRIVE_FOLDER = '/Users/sbezawada/Documents/Workspace/MLE-COURSE/Capstone/Data/true/'
entries = [entry for entry in os.listdir(DRIVE_FOLDER) if os.path.isfile(os.path.join(DRIVE_FOLDER, entry))]
true_dict = {'name':[]}
for file in entries:

    true_dict['name'].append(file)


In [4]:
df1 = pd.DataFrame(false_dict)
df1['label'] = 0
df1['path'] = '/Users/sbezawada/Documents/Workspace/MLE-COURSE/Capstone/Data/false/'

df2 = pd.DataFrame(true_dict)
df2['label'] = 1
df2['path'] = '/Users/sbezawada/Documents/Workspace/MLE-COURSE/Capstone/Data/true/'

# combine both dataframes
df = pd.concat([df1,df2])
#shuffle the labels
df = df.sample(frac=1, random_state=56).reset_index(drop=True)

print('The length of the dataset is: ', f'{df.shape[0]:,}')

labels = df['label'].unique()
df.head

The length of the dataset is:  1,567


<bound method NDFrame.head of                    name  label  \
0     Mani_false582.wav      0   
1           Mani350.wav      1   
2     Mani_false567.wav      0   
3     Mani_false854.wav      0   
4      Mani_false25.wav      0   
...                 ...    ...   
1562        Mani606.wav      1   
1563        Mani329.wav      1   
1564         Mani96.wav      1   
1565   Mani_false91.wav      0   
1566  Mani_false564.wav      0   

                                                   path  
0     /Users/sbezawada/Documents/Workspace/MLE-COURS...  
1     /Users/sbezawada/Documents/Workspace/MLE-COURS...  
2     /Users/sbezawada/Documents/Workspace/MLE-COURS...  
3     /Users/sbezawada/Documents/Workspace/MLE-COURS...  
4     /Users/sbezawada/Documents/Workspace/MLE-COURS...  
...                                                 ...  
1562  /Users/sbezawada/Documents/Workspace/MLE-COURS...  
1563  /Users/sbezawada/Documents/Workspace/MLE-COURS...  
1564  /Users/sbezawada/Documents/Worksp

In [5]:
#data check if new records made it in
df[df['name'] =='Mani_false881.wav']

,name,label,path
727,Mani_false881.wav,0,/Users/sbezawada/Documents/Workspace/MLE-COURS...


In [6]:
#extract for the complete dataset
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    #set a window width to 25 ms and the stride to 10 ms
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40,hop_length=int(0.010*sample_rate), n_fft=int(0.025*sample_rate))
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    return mfccs_scaled_features

In [7]:
extracted_features=[]
for i in range(len(df)):
    #print(i)
    file_name = df['path'][i] + df['name'][i] #os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=df['label'][i]
    #print(final_class_labels)
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [8]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-472.0259, 145.97658, 3.9457073, 36.730343, 1...",0
1,"[-360.56946, 146.80739, 17.688366, 42.45998, 5...",1
2,"[-511.9074, 104.88985, 16.42294, 9.589837, 7.8...",0
3,"[-487.00116, 108.75475, 38.73437, 38.12563, 17...",0
4,"[-666.19836, 55.971718, -25.079172, 51.797554,...",0


In [9]:
extracted_features_df['class'].value_counts()

0    791
1    776
Name: class, dtype: int64

In [10]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
X.shape

(1567, 40)

In [11]:
### Train Test Split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
X_train.shape

(1253, 40)

In [12]:
X_train2 = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_train2.shape

(1253, 1, 40)

In [19]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
tf.random.set_seed(7)
from sklearn.model_selection import GridSearchCV

from scikeras.wrappers import KerasClassifier

2.10.0


In [20]:
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
n_layers = [ 5, 10, 15, 20, 25, 30, 60, 100, 150, 200]
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum,n_layers=n_layers)

batch_size =500
#vanilla LSTM
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=60,return_sequences=True,input_shape=(X_train2.shape[1], X_train2.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(200,activation=tf.nn.relu))
model.add(Dense(1, activation=tf.nn.sigmoid))
model.compile(loss='binary_crossentropy',metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

keras_clf = KerasClassifier(model = model, optimizer="adam", epochs=100, verbose=0)

grid = GridSearchCV(estimator=keras_clf, param_grid=param_grid, n_jobs=-1,scoring="accuracy")
grid_result = grid.fit(X_train2,y_train.reshape((-1,1)),epochs=50,batch_size=batch_size,verbose=0,validation_split=0.25)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


INFO:tensorflow:Assets written to: ram:///var/folders/mt/52sww7c50ws851nqq95j74qc0000gp/T/tmpraj5w_br/assets


INFO:tensorflow:Assets written to: ram:///var/folders/mt/52sww7c50ws851nqq95j74qc0000gp/T/tmpraj5w_br/assets


INFO:tensorflow:Assets written to: ram:///var/folders/mt/52sww7c50ws851nqq95j74qc0000gp/T/tmpqpyb1e_n/assets


INFO:tensorflow:Assets written to: ram:///var/folders/mt/52sww7c50ws851nqq95j74qc0000gp/T/tmpqpyb1e_n/assets


INFO:tensorflow:Assets written to: ram:///var/folders/mt/52sww7c50ws851nqq95j74qc0000gp/T/tmpi3ag55w9/assets


...

INFO:tensorflow:Assets written to: ram:///var/folders/mt/52sww7c50ws851nqq95j74qc0000gp/T/tmp731ejlcd/assets


INFO:tensorflow:Assets written to: ram:///var/folders/mt/52sww7c50ws851nqq95j74qc0000gp/T/tmp731ejlcd/assets


INFO:tensorflow:Assets written to: ram:///var/folders/mt/52sww7c50ws851nqq95j74qc0000gp/T/tmpdkzbylfg/assets


INFO:tensorflow:Assets written to: ram:///var/folders/mt/52sww7c50ws851nqq95j74qc0000gp/T/tmpdkzbylfg/assets


ValueError: 
All the 1500 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1500 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sbezawada/.virtualenvs/Capstone2/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sbezawada/.virtualenvs/Capstone2/lib/python3.9/site-packages/scikeras/wrappers.py", line 1494, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/sbezawada/.virtualenvs/Capstone2/lib/python3.9/site-packages/scikeras/wrappers.py", line 762, in fit
    self._fit(
  File "/Users/sbezawada/.virtualenvs/Capstone2/lib/python3.9/site-packages/scikeras/wrappers.py", line 918, in _fit
    X, y = self._i

In [27]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []

    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment

    return layers

def createmodel():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=60,return_sequences=True,input_shape=(X_train2.shape[1], X_train2.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(200,activation=tf.nn.relu))
    model.add(Dense(1, activation=tf.nn.sigmoid))

    return model

model =  KerasClassifier(model=createmodel, loss='binary_crossentropy',metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],verbose = False)

In [28]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=3)

grid_result = grid.fit(X_train2,y_train.reshape((-1,1)),epochs=50)

TypeError: cannot pickle '_thread._local' object

keras tuner
optuna- framework uses bayesian sampling
cross validation